## Final Project - Yelp Data Analysis
### Big Data Machine Learning - PySpark

Members:

**1. Rolamjaya Hotmartua**

**2. YiChin Tzou**

**3. Zoey Chen**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import warnings
warnings.filterwarnings("ignore")

In [69]:
from pyspark.context import SparkContext
from graphframes import *

ModuleNotFoundError: No module named 'graphframes'

In [2]:
#create Spark session
spark = SparkSession.builder.appName('YelpBigDataML').config("spark.jars.packages", "graphframes:graphframes:0.8.1-spark2.4-s_2.11").getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/27 04:30:27 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/02/27 04:30:27 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/02/27 04:30:28 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/02/27 04:30:28 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


[('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.yarn.historyServer.address',
  'hub-msca-bdp-dphub-student-rhotmartua-m:18080'),
 ('spark.dataproc.sql.joinConditionReorder.enabled', 'true'),
 ('spark.executor.memory', '5g'),
 ('spark.dataproc.sql.local.rank.pushdown.enabled', 'true'),
 ('spark.driver.port', '33067'),
 ('spark.yarn.unmanagedAM.enabled', 'true'),
 ('spark.sql.autoBroadcastJoinThreshold', '43m'),
 ('spark.history.fs.logDirectory',
  'gs://dataproc-temp-us-central1-635155370842-uzamlpgc/e989acb7-641c-491d-a9c8-9bbc0984dd05/spark-job-history'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.driver.appUIAddress',
  'http://hub-msca-bdp-dphub-student-rhotmartua-m.c.msca-bdp-student-ap.internal:35813'),
 ('spark.metrics.namespace',
  'app_name:${spark.app.name}.app_id:${spark.app.id}'),
 ('spark.dataproc.metrics.listener.metrics.collecto

### 1) Data Exploration

#### a. Read the movie synopsis data into Spark   

In [42]:
df_user = spark.read\
    .json("gs://msca-bdp-student-gcs/group3_final/yelp_academic_dataset_user.json" )

In [43]:
df_user.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [44]:
df_review = spark.read\
    .json("gs://msca-bdp-student-gcs/group3_final/yelp_academic_dataset_review.json" )

In [45]:
df_review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [48]:
df_business = spark.read\
    .json("gs://msca-bdp-student-gcs/group3_final/yelp_academic_dataset_business.json" )

23/02/27 05:15:50 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [49]:
df_business.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [50]:
df_checkin = spark.read\
    .json("gs://msca-bdp-student-gcs/group3_final/yelp_academic_dataset_checkin.json" )

In [51]:
df_checkin.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)



In [52]:
df_tip = spark.read\
    .json("gs://msca-bdp-student-gcs/group3_final/yelp_academic_dataset_tip.json" )

In [53]:
df_tip.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



In [46]:
df_user.show(5)

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|  name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+------+------------+------+--------------------+-------------------+
|         3.91|    

In [47]:
df_review.show(5)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [54]:
df_business.show(5)

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|Santa Barbara|                null|      0|34.4266787|-119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|       Affton|{8:0-18:30, 0:0-0...|      1| 38.551126|  -90.335695|    

In [55]:
df_checkin.show(5)

+--------------------+--------------------+
|         business_id|                date|
+--------------------+--------------------+
|---kPU91CF4Lq2-Wl...|2020-03-13 21:10:...|
|--0iUa4sNDFiZFrAd...|2010-09-13 21:43:...|
|--30_8IhuyMHbSOcN...|2013-06-14 23:29:...|
|--7PUidqRWpRSpXeb...|2011-02-15 17:12:...|
|--7jw19RH9JKXgFoh...|2014-04-21 20:42:...|
+--------------------+--------------------+
only showing top 5 rows



In [56]:
df_tip.show(5)

+--------------------+----------------+-------------------+--------------------+--------------------+
|         business_id|compliment_count|               date|                text|             user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
|3uLgwr0qeCNMjKenH...|               0|2012-05-18 02:17:21|Avengers time wit...|AGNUgVwnZUey3gcPC...|
|QoezRbYQncpRqyrLH...|               0|2013-02-05 18:35:10|They have lots of...|NBN4MgHP9D3cw--Sn...|
|MYoRNLb5chwjQe3c_...|               0|2013-08-18 00:56:08|It's open even wh...|-copOvldyKh1qr-vz...|
|hV-bABTK-glh5wj31...|               0|2017-06-27 23:05:38|Very decent fried...|FjMQVZjSqY8syIO-5...|
|_uN0OudeJ3Zl_tf6n...|               0|2012-10-06 19:43:09|Appetizers.. plat...|ld0AperBXk1h6Ubqm...|
+--------------------+----------------+-------------------+--------------------+--------------------+
only showing top 5 rows



### Create Graph

In [66]:
#Create dataframes for vertices and edges 
cols_to_drop = ["Friends"]
vertices = df_user.drop(*cols_to_drop)

from pyspark.sql.functions import split, explode, col,trim,lit

df_friends_vert = df.select(["user_id","friends"]) 

df_expand = df_friends_vert.withColumn("friends", split(col("friends"), ","))  # split the friends column by comma
df_expand = df_expand.select(col("*"), explode(col("friends")).alias("friend"))  # explode the friend column

# remove leading and trailing whitespace from "my_column"
df_expand = df_expand.withColumn("friend", trim(df_expand["friend"]))

df_expand = df_expand.withColumn("relationship", lit("friends"))

cols_to_drop = ["Friends"]
df_expand = df_expand.drop(*cols_to_drop)

edges = df_expand

In [63]:
vertices.show(5)

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+-----+------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|funny|  name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+-----+------+------------+------+--------------------+-------------------+
|         3.91|            467|             56|             467|           250|   

In [67]:
edges.show(5, truncate=False)

+----------------------+----------------------+------------+
|user_id               |friend                |relationship|
+----------------------+----------------------+------------+
|qVc8ODYU5SZjKXVBgXdI7w|NSCy54eWehBJyZdG2iE84w|friends     |
|qVc8ODYU5SZjKXVBgXdI7w|pe42u7DcCH2QmI81NX-8qA|friends     |
|qVc8ODYU5SZjKXVBgXdI7w|EjlCGf14tYMPJ0rsrL703w|friends     |
|qVc8ODYU5SZjKXVBgXdI7w|7OagHIAKx2Rm9z3CQ0OmvA|friends     |
|qVc8ODYU5SZjKXVBgXdI7w|uZoFvKs0dahfffIqHQf1eA|friends     |
+----------------------+----------------------+------------+
only showing top 5 rows



In [68]:
#create a graph using the vertices and edges dataframes
g = GraphFrame(vertices, edges)

NameError: name 'GraphFrame' is not defined

In [ ]:
## Show vertices
g.vertices.show()

### Explore Graph

In [ ]:
## Check the number of edges
g.edges.show()

In [ ]:
## Check the number of edges of each vertex
g.degrees.show()

In [ ]:
#The incoming degree of the vertices
g.inDegrees.show()

In [ ]:
#The outgoing degree of the vertices
g.outDegrees.show()

### Connected Components

A connected component of a graph is a subgraph in which any two vertices are connected to each other by one or more edges, and which is connected to no additional vertices in the supergraph. Connected components detection can be interesting for clustering, but also to make your computations more efficient. 

Practically, GraphFrames requires you to set a directory where it can save checkpoints. Create such a folder in your working directory and drop the following line (where graphframes_cps is your new folder) in Jupyter to set the checkpoint directory.

The number of connected components for directed and undirected graphs are different.

Our mini-graph has two connected components, which are described for each vertex in the component column.

In [ ]:
## Display all connected components
sc = spark.sparkContext
sc.setCheckpointDir('graphframes_cps')
g.connectedComponents().show()

In [ ]:
result = g.stronglyConnectedComponents(maxIter=10)
result.select("id", "component").show()

### Filter Graph

A GraphFrame itself can’t be filtered, but DataFrames deducted from a Graph can. Consequently, the filter-function (or any other function) can be used just as you would use it with DataFrames. Pay attention to the use of quotation marks in filter criteria.

In [ ]:
## Filter vertices to show people older than 30 yrs
g.vertices.filter("age > 30").show()

In [ ]:
## Filter and sort vertices with degree >=2
g.inDegrees.filter("inDegree >= 2").sort("inDegree", ascending=False).show()

In [ ]:
## Filter and sort edges of type friend
g.edges.filter('type == "friend"').show()

In [ ]:
## Filter and sort edges of type friend
g.edges.filter('type == "follows"').show()

### Motif Finding

Finding motifs helps to execute queries to discover structural patterns in graphs. 

Network motifs are patterns that occur repeatedly in the graph and represent the relationships between the vertices. GraphFrames motif finding uses a declarative Domain Specific Language (DSL) for expressing structural queries. 

The query can be invoked by using the find-function, where the motif (in quotation marks) is expressed as the first parameter of the function.

The following example will search for pairs of vertices a,b connected by edge e and pairs of vertices b,c connected by edge e2. It will return a DataFrame of all such structures in the graph, with columns for each of the named elements (vertices or edges) in the motif.

In [ ]:
g.find("(a)-[e]->(b); (b)-[e2]->(a)").show(truncate = False)

In [ ]:
#To query all the mutual friends between 2 and 3 we can filter the DataFrame

mutualFriends =  g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(b); (b)-[]->(a)").dropDuplicates()
mutualFriends.filter('a.id == 2 and c.id == 3').show(truncate = False)
#mutualFriends.show(truncate = False)

### TriangleCount 

TriangleCount counts the number of triangles passing through each vertex in this graph. A triangle can be defined as a group of three vertices that is interrelated, i.e. a has an edge to b, b has an edge to c, and c has an edge to a. 

Triangles are used for various tasks for real‐life networks, including community discovery, link prediction, and spam filtering.

<img width="30%" src="https://miro.medium.com/max/791/0*MGZV-XXQl9YN_fwD.png"/>

In [ ]:
g.triangleCount().show()

### PageRank

PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites.

The PageRank algorithm holds that an imaginary surfer who is randomly clicking on links will eventually stop clicking. The probability, at any step, that the person will continue is a damping factor. The damping factor can be be set by changing the resetProbability parameter. Other important parameters are the tolerance (tol) and the maximum number of iterations (maxIter).

https://en.wikipedia.org/wiki/PageRank

PageRank measures the importance of each vertex in a graph, assuming an edge from u to v represents an endorsement of v’s importance by u. 

ϵ is the reset probability ( 1 − ϵ } is also called as damping factor used in the PageRank computation.

Identify important vertices in a graph based on connections.

In [ ]:
pr = g.pageRank(resetProbability=0.15, tol=0.01)## look at the pagerank score for every vertex
pr.vertices.show()

## look at the weight of every edge
pr.edges.show()

## Label Propagation

Run static Label Propagation Algorithm for detecting communities in networks.

Each node in the network is initially assigned to its own community. At every superstep, nodes send their community affiliation to all neighbors and update their state to the most frequent community affiliation of incoming messages.

LPA is a standard community detection algorithm for graphs. It is very inexpensive computationally, although (1) convergence is not guaranteed and (2) one can end up with trivial solutions (all nodes are identified into a single community).

In [ ]:
result = g.labelPropagation(maxIter=5)
result.show()

### BFS

In [ ]:
paths = g.bfs("name = 'Banks'", "age > 40")
paths.show()